In [2]:
drive_path = 'c:/'
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import ks_2samp
from scipy.stats import anderson_ksamp
from scipy.stats import kruskal
from scipy.stats import variation
from scipy import signal as sps
import seaborn as sns
import glob
import re

# Detrended peaks

In [3]:
# (pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\160330_3\\44dtnew.txt'))
# badfile=(pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\160330_3\\44dt.txt'))
# rightfile=badfile.iloc[:,1:16]
# rightfile
# rightfile.to_csv('44dtnew.txt')

In [3]:
date='160325_1'
os.chdir('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\%s'%date)

In [3]:
#This piece spits out all the peaks from one session in one dataframe
peakdf=pd.DataFrame([])
date='160626_2'
os.chdir('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\%s'%date)
for filename in glob.glob('*dt.txt'):
    f=pd.read_csv(filename,nrows=175)
    df=f[[col for col in f.columns if 'G PMT' in col]]
    peak=[]
    for col in df.columns:
        a=df[col]
        firsta=1;
        firstb=24;
        #Figures out if there is a min or max and sees if it passes threshold (3SD)
        if np.absolute(min(a[26:80]))>np.absolute(max(a[26:80])) and np.absolute(min(a[26:80]))>=3*np.std(df[col][firsta:firstb]):
            b=min(a[26:80])
            peak.append(b)
        elif np.absolute(max(a[26:80]))>np.absolute(min(a[26:80]))and np.absolute(max(a[26:80]))>=3*np.std(df[col][firsta:firstb]):
            b=max(a[26:80])
            peak.append(b)
        else:
            b=0
            peak.append(b)
        peaks=pd.DataFrame(peak).T
    peaks.columns=df.columns
    peaks=pd.concat([pd.DataFrame({'Trial':[int(filename.split('dt')[0])]}),peaks],axis=1)
    peakdf=peakdf.append(peaks,ignore_index=True)
# peakdf.to_csv('%s_peaks.csv'%date,index=False)

In [4]:
peakdf

,Trial,G PMT (1),G PMT (2),G PMT (3),G PMT (4),G PMT (5),G PMT (6),G PMT (7),G PMT (8),G PMT (9),...,G PMT (17),G PMT (18),G PMT (19),G PMT (20),G PMT (21),G PMT (22),G PMT (23),G PMT (24),G PMT (25),G PMT (26)
0,10,1.348483,0.827338,2.066554,0.973768,1.256708,1.094561,1.970355,2.170065,1.383174,...,0.986188,1.353452,1.652477,1.771158,1.289377,2.594698,2.606290,1.817215,1.749630,2.178384
1,11,0.880285,0.575757,1.365327,0.600886,0.741207,0.774862,1.273472,1.497569,0.871895,...,0.567321,0.896947,1.214118,1.301672,0.960139,1.576790,1.482137,1.301819,1.142405,1.426847
2,12,0.564135,0.432287,0.917244,0.455624,0.548823,0.565288,0.970520,1.069985,0.801432,...,0.361561,0.623287,0.891940,0.883235,0.625039,0.971943,1.306090,0.903109,0.909851,1.006884
3,13,1.219061,0.671587,1.465497,0.855079,1.050465,1.135557,1.411054,1.329367,1.021675,...,0.728894,1.280372,1.345380,1.402337,1.110819,1.670811,1.369600,1.734945,1.180116,1.537261
4,14,2.270378,1.206907,3.436765,1.771767,1.794232,1.982638,2.546833,3.237607,1.975369,...,1.640651,2.187905,2.456511,3.230087,2.621568,3.384514,2.835851,3.122166,2.153089,2.788357
5,15,1.535268,0.870842,2.313791,0.985821,1.101559,1.030359,1.887143,2.537820,2.067564,...,0.979901,1.184880,1.795884,2.079371,1.484679,2.469555,2.874112,2.254843,1.910691,2.240307
6,16,0.892533,0.602725,1.243088,0.615415,0.671238,0.793600,1.112826,1.212784,1.208369,...,0.496358,0.689027,1.132382,1.155144,0.795847,1.304345,1.493539,1.286426,1.045183,1.343541
7,17,1.380346,0.803607,1.821771,1.076577,1.076692,1.273785,1.727817,1.821554,1.224598,...,0.907855,1.291226,1.511625,1.912853,1.511792,1.952347,1.720925,1.790391,1.329803,1.741874
8,18,0.513968,0.381075,0.713474,0.362015,0.420011,0.444047,0.699946,0.738189,0.719339,...,0.266018,0.371043,0.735623,0.712813,0.469944,0.834711,0.990126,0.707138,0.624644,0.813339
9,19,1.083280,0.746743,1.522384,0.842612,0.946848,0.904668,1.472708,1.562746,1.719422,...,0.685992,0.983026,1.473926,1.471597,1.088801,1.709548,1.998244,1.765468,1.399827,1.780926


In [4]:
trials=pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\Analysis\\Odor_Panel\\Odor_Trials.csv')
filerow=trials.loc[trials['File']==date]
odortrials={}
for t in trials.Odor.unique():
    y={t:[int(x) for x in filerow.loc[filerow['Odor'] == t][['T1','T2','T3','T4']].values.tolist()[0]]}
    odortrials.update(y)

In [6]:
#Get average peak across all trials using peakdf dataframe
meandf=pd.DataFrame([])
for key in odortrials:
    odor=odortrials[key]
    mean=[]
    for col in peakdf.loc[peakdf['Trial'].isin(odor)][[col for col in peakdf.loc[peakdf['Trial'].isin(odor)].columns if 'G PMT' in col]]:
        mean.append(peakdf.loc[peakdf['Trial'].isin(odor)][col].mean())
    mean=pd.DataFrame(mean).T
    mean.columns=peakdf.loc[peakdf['Trial'].isin(odor)][[col for col in peakdf.loc[peakdf['Trial'].isin(odor)].columns if 'G PMT' in col]].columns
    meandf=meandf.append(mean)
meandf=meandf.reset_index(drop=True)
meandf.columns=[str(col)+'_'+date for col in meandf.columns]
meandf=pd.concat([pd.DataFrame({'Odor':odortrials.keys()}),meandf],axis=1)
# meandf.to_csv('%s_mean.csv'%date,index=False)

In [7]:
#Get proportion of successful trials
successdf=pd.DataFrame([])
for key in odortrials:
    odor=odortrials[key]
    newdf=peakdf.loc[peakdf['Trial'].isin(odor)]
    s=[]
    for col in peakdf.loc[peakdf['Trial'].isin(odor)][[col for col in peakdf.loc[peakdf['Trial'].isin(odor)].columns if 'G PMT' in col]]:
        s.append(np.divide((newdf.loc[:,col]!=0).sum(),float(len(newdf.loc[:,col]))))
    s=pd.DataFrame(s).T
    s.columns=peakdf.loc[peakdf['Trial'].isin(odor)][[col for col in peakdf.loc[peakdf['Trial'].isin(odor)].columns if 'G PMT' in col]].columns
    successdf=successdf.append(s)
successdf=successdf.reset_index(drop=True)
successdf.columns=[str(col)+'_'+date for col in successdf.columns]
successdf=pd.concat([pd.DataFrame({'Odor':odortrials.keys()}),successdf],axis=1)
successdf.to_csv('%s_success.csv'%date,index=False)

# Detrended integral

In [8]:
# # date='160330_3'
# # os.chdir('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\%s'%date)
# #example dataframe
# for filename in 
# df=pd.read_csv('1dt.txt')
# df=df[[col for col in df.columns if 'G PMT' in col]]

In [9]:
temp=pd.DataFrame([])
date='160330_3'
os.chdir('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\%s'%date)
#Pull the trials that correspond to specific date/odors
trials=pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\Analysis\\Odor_Panel\\Odor_Trials.csv')
filerow=trials.loc[trials['File']==date]
odortrials={}
for t in trials.Odor.unique():
    y={t:[int(x) for x in filerow.loc[filerow['Odor'] == t][['T1','T2','T3','T4']].values.tolist()[0]]}
    odortrials.update(y)
#Get the frame rate for a specified date
num=trials.File.unique().tolist().index('%s'%date)
fr=trials.loc[trials['File']==trials.File.unique().tolist()[num]]['FrameRate'].iloc[0]
#Get the integral
intdf=pd.DataFrame([])
for filename in glob.glob('*dt.txt'):
    f=pd.read_csv(filename,nrows=125)
    df=f[[col for col in f.columns if 'G PMT' in col]]
    winstart=np.int(4*fr)
    winend=np.int(12*fr)
    integral=[]
    for col in df.columns:
        a=df[col]
        firsta=1;
        firstb=24;
        #Figures out if there is a min or max and sees if it passes threshold (3SD)
        if np.absolute(min(a[26:80]))>np.absolute(max(a[26:80])) and np.absolute(min(a[26:80]))>=3*np.std(df[col][firsta:firstb]):
            b=sum(df[col][winstart:winend]*(1/fr))
            integral.append(b)
        elif np.absolute(max(a[26:80]))>np.absolute(min(a[26:80]))and np.absolute(max(a[26:80]))>=3*np.std(df[col][firsta:firstb]):
            b=sum(df[col][winstart:winend]*(1/fr))
            integral.append(b)
        else:
            b=0
            integral.append(b)
    integral=pd.DataFrame(integral).T
    integral.columns=df.columns
    integral=pd.concat([pd.DataFrame({'Trial':[int(filename.split('dt')[0])]}),integral],axis=1)
    intdf=intdf.append(integral)
# intdf.to_csv('%s_integral.csv'%date,index=False)

In [10]:
#Get average integral across all trials using integral dataframe
meanint=pd.DataFrame([])
for key in odortrials:
    odor=odortrials[key]
    mean=[]
    for col in intdf.loc[intdf['Trial'].isin(odor)][[col for col in intdf.loc[intdf['Trial'].isin(odor)].columns if 'G PMT' in col]]:
        mean.append(intdf.loc[intdf['Trial'].isin(odor)][col].mean())
    mean=pd.DataFrame(mean).T
    mean.columns=intdf.loc[intdf['Trial'].isin(odor)][[col for col in intdf.loc[intdf['Trial'].isin(odor)].columns if 'G PMT' in col]].columns
    meanint=meanint.append(mean)
meanint=meanint.reset_index(drop=True)
meanint.columns=[str(col)+'_'+date for col in meanint.columns]
meanint=pd.concat([pd.DataFrame({'Odor':odortrials.keys()}),meanint],axis=1)
# meanint.to_csv('%s_meanint.csv'%date,index=False)

# Detrended baseline

In [11]:
temp=pd.DataFrame([])
date='160330_3'
os.chdir('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\%s'%date)
#Pull the trials that correspond to specific date/odors
trials=pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\Analysis\\Odor_Panel\\Odor_Trials.csv')
filerow=trials.loc[trials['File']==date]
odortrials={}
for t in trials.Odor.unique():
    y={t:[int(x) for x in filerow.loc[filerow['Odor'] == t][['T1','T2','T3','T4']].values.tolist()[0]]}
    odortrials.update(y)
#Get the frame rate for a specified date
num=trials.File.unique().tolist().index('%s'%date)
fr=trials.loc[trials['File']==trials.File.unique().tolist()[num]]['FrameRate'].iloc[0]
#Get baseline
baseline=pd.DataFrame([])
for filename in glob.glob('*dt.txt'):
    f=pd.read_csv(filename,nrows=125)
    df=f[[col for col in f.columns if 'G PMT' in col]]
    winstart=np.int(4*fr)
    winend=np.int(12*fr)
    base=[]
    for col in df.columns:
        a=df[col]
        firsta=1;
        firstb=24;
        b=(df[col][firsta:firstb]).mean()
        base.append(b)
    base=pd.DataFrame(base).T
    base.columns=df.columns
    base=pd.concat([pd.DataFrame({'Trial':[int(filename.split('dt')[0])]}),base],axis=1)
    baseline=baseline.append(base)
# baseline.to_csv('%s_baseline.csv'%date,index=False)

In [36]:
baseline

,Trial,G PMT (1),G PMT (2),G PMT (3),G PMT (4),G PMT (5),G PMT (6),G PMT (7),G PMT (8),G PMT (9),G PMT (10),G PMT (11),G PMT (12),G PMT (13),G PMT (14),G PMT (15)
0,10,-0.001772,-0.004882,-0.007575,0.002221,-0.001303,0.000239,-0.010954,-0.008220,-0.002090,-0.005447,-0.005013,-0.005016,-0.005820,-0.005131,-0.006786
0,11,-0.007566,-0.005516,-0.010814,0.000050,-0.004390,-0.001933,-0.000676,-0.009076,0.004436,-0.004560,-0.009323,-0.011845,-0.003606,-0.003959,-0.005751
0,12,-0.008305,-0.004887,-0.004887,0.000445,-0.003743,0.002659,0.004728,-0.002949,0.003973,-0.020656,-0.003786,-0.008455,-0.003549,-0.005309,-0.004675
0,13,-0.003940,-0.003201,-0.005427,-0.003443,-0.004063,-0.004560,-0.002286,-0.004222,-0.004502,0.005844,-0.008771,-0.003226,-0.002800,-0.005256,-0.009917
0,14,-0.001257,0.001370,0.001014,-0.002701,-0.005413,-0.000309,-0.001427,0.004934,-0.000473,0.005222,-0.001874,-0.004310,-0.000015,-0.007021,-0.002977
0,15,0.000266,-0.000219,-0.007417,0.011546,-0.005948,0.002980,0.000988,-0.007289,0.000278,-0.007043,-0.007277,0.002790,-0.011454,-0.008510,0.001835
0,16,-0.007216,-0.005283,-0.008189,-0.006310,-0.003022,-0.005340,-0.007450,-0.007774,-0.010824,0.008362,-0.003605,-0.002658,-0.002769,-0.003603,-0.000631
0,17,0.004017,-0.002160,-0.002432,0.005402,-0.001335,0.000963,-0.000760,-0.007520,-0.000687,-0.001767,-0.004742,-0.002614,-0.003313,-0.011488,-0.000444
0,18,-0.005194,-0.004806,-0.002486,0.000830,-0.002064,0.005237,-0.001066,-0.004164,0.003371,0.010896,-0.003461,-0.007090,-0.002751,-0.004746,-0.003140
0,19,-0.006556,-0.003986,0.001115,-0.000171,0.003068,0.006268,0.000263,0.010030,0.009705,0.007040,-0.001674,-0.002352,0.000548,-0.002654,0.004113


In [12]:
#mean baseline
meanbase=pd.DataFrame([])
for key in odortrials:
    odor=odortrials[key]
    mean=[]
    for col in baseline.loc[baseline['Trial'].isin(odor)][[col for col in baseline.loc[baseline['Trial'].isin(odor)].columns if 'G PMT' in col]]:
        mean.append(baseline.loc[baseline['Trial'].isin(odor)][col].mean())
    mean=pd.DataFrame(mean).T
    mean.columns=baseline.loc[baseline['Trial'].isin(odor)][[col for col in baseline.loc[baseline['Trial'].isin(odor)].columns if 'G PMT' in col]].columns
    meanbase=meanbase.append(mean)
meanbase=meanbase.reset_index(drop=True)
meanbase.columns=[str(col)+'_'+date for col in meanbase.columns]
meanbase=pd.concat([pd.DataFrame({'Odor':odortrials.keys()}),meanbase],axis=1)
# meanbase.to_csv('%s_meanbase.csv'%date,index=False)

In [38]:
meanbase

,Odor,G PMT (1)_160330_3,G PMT (2)_160330_3,G PMT (3)_160330_3,G PMT (4)_160330_3,G PMT (5)_160330_3,G PMT (6)_160330_3,G PMT (7)_160330_3,G PMT (8)_160330_3,G PMT (9)_160330_3,G PMT (10)_160330_3,G PMT (11)_160330_3,G PMT (12)_160330_3,G PMT (13)_160330_3,G PMT (14)_160330_3,G PMT (15)_160330_3
0,IAA10,-0.004230,0.000723,-0.000525,0.006939,0.004007,0.004996,0.008156,-0.002754,0.007712,-0.006166,0.001741,-0.003249,-0.000863,-0.001321,0.000379
1,IAA01,0.000210,-0.002284,-0.002584,0.005723,0.000540,-0.000209,0.004656,-0.002402,-0.001625,-0.005141,-0.003696,-0.003886,-0.001328,-0.005061,-0.001324
2,AP,-0.006349,-0.002911,-0.000479,0.000938,-0.003350,-0.002714,-0.002455,-0.003111,-0.002183,-0.000381,-0.002530,-0.005367,-0.001410,-0.003097,-0.002435
3,MS10,-0.003141,-0.003885,-0.003913,-0.003040,-0.001047,0.000144,0.000143,0.000346,-0.002007,-0.002806,-0.002141,-0.003498,-0.001442,-0.005421,-0.003581
4,IAA05,-0.001864,-0.002353,-0.005286,0.002599,-0.001782,-0.000211,0.003648,-0.004936,0.000865,0.003699,-0.000824,-0.003090,-0.002415,-0.004693,-0.000452
5,Hexanal01,-0.005869,-0.003338,-0.003011,-0.000483,0.000115,0.001698,0.000356,0.005437,0.004524,0.004547,-0.003022,-0.004029,0.001947,-0.001096,-0.002302
6,EB,-0.000458,-0.001112,-0.003886,0.004596,-0.000258,0.004115,0.002212,-0.001196,0.003517,-0.000141,-0.003145,-0.001224,-0.004046,-0.006046,0.003135
7,MS01,-0.004528,-0.000533,-0.001770,-0.003275,-0.003056,0.000108,0.000719,-0.003364,-0.002029,0.000888,-0.003760,-0.002115,-0.000040,0.000084,-0.001479
8,PA,-0.002716,-0.004157,-0.007559,0.004096,0.000683,-0.000955,0.002639,-0.000913,-0.002134,-0.000056,0.000548,-0.001705,-0.003881,-0.005284,0.000902
9,MS05,-0.005032,-0.002676,-0.003452,-0.000889,-0.000047,0.000391,0.003618,-0.001625,-0.001405,-0.001851,-0.002419,-0.002895,-0.000864,-0.002235,-0.002714


In [4]:
# test=pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\160321_1\\160321_1_baseline.csv')

In [5]:
# plt.plot(test.iloc[:,1:-1]);

In [6]:
# test2=pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\160321_1\\160321_1_meanbase.csv')

In [7]:
# plt.plot(test2.iloc[:,1:]);

In [8]:
# test2

# Untrended baseline dataframe

In [5]:
temp=pd.DataFrame([])
date='160626_2'
os.chdir('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\Untrended_For_B\\%s'%date)
#Pull the trials that correspond to specific date/odors
trials=pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\Analysis\\Odor_Panel\\Odor_Trials.csv')
filerow=trials.loc[trials['File']==date]
odortrials={}
for t in trials.Odor.unique():
    y={t:[int(x) for x in filerow.loc[filerow['Odor'] == t][['T1','T2','T3','T4']].values.tolist()[0]]}
    odortrials.update(y)
#Get the frame rate for a specified date
num=trials.File.unique().tolist().index('%s'%date)
fr=trials.loc[trials['File']==trials.File.unique().tolist()[num]]['FrameRate'].iloc[0]
#Get baseline
baseline=pd.DataFrame([])
for filename in glob.glob('*.txt'):
    f=pd.read_table(filename,skiprows=4)
    df=f[[col for col in f.columns if 'G PMT (' in col]]
    base=[]
    for col in df.columns:
        a=df[col]
        firsta=2;
        firstb=24;
        b=(df[col][firsta:firstb]).mean()
        base.append(b)
    base=pd.DataFrame(base).T
    base.columns=df.columns
    base=pd.concat([pd.DataFrame({'Trial':[int(filename.split('.txt')[0])]}),base],axis=1)
    baseline=baseline.append(base)
baseline.to_csv('%s_untrendedbaseline.csv'%date,index=False)

#mean baseline
meanbase=pd.DataFrame([])
for key in odortrials:
    odor=odortrials[key]
    mean=[]
    for col in baseline.loc[baseline['Trial'].isin(odor)][[col for col in baseline.loc[baseline['Trial'].isin(odor)].columns if 'G PMT' in col]]:
        mean.append(baseline.loc[baseline['Trial'].isin(odor)][col].mean())
    mean=pd.DataFrame(mean).T
    mean.columns=baseline.loc[baseline['Trial'].isin(odor)][[col for col in baseline.loc[baseline['Trial'].isin(odor)].columns if 'G PMT' in col]].columns
    meanbase=meanbase.append(mean)
meanbase=meanbase.reset_index(drop=True)
meanbase.columns=[str(col)+'_'+date for col in meanbase.columns]
meanbase=pd.concat([pd.DataFrame({'Odor':odortrials.keys()}),meanbase],axis=1)
meanbase.to_csv('%s_meanbase.csv'%date,index=False)

In [6]:
#mean baseline
meanbase=pd.DataFrame([])
for key in odortrials:
    odor=odortrials[key]
    mean=[]
    for col in baseline.loc[baseline['Trial'].isin(odor)][[col for col in baseline.loc[baseline['Trial'].isin(odor)].columns if 'G PMT' in col]]:
        mean.append(baseline.loc[baseline['Trial'].isin(odor)][col].mean())
    mean=pd.DataFrame(mean).T
    mean.columns=baseline.loc[baseline['Trial'].isin(odor)][[col for col in baseline.loc[baseline['Trial'].isin(odor)].columns if 'G PMT' in col]].columns
    meanbase=meanbase.append(mean)
meanbase=meanbase.reset_index(drop=True)
meanbase.columns=[str(col)+'_'+date for col in meanbase.columns]
meanbase=pd.concat([pd.DataFrame({'Odor':odortrials.keys()}),meanbase],axis=1)
# meanbase.to_csv('%s_meanbase.csv'%date,index=False)

In [8]:
meanbase

,Odor,G PMT (1)_160626_2,G PMT (2)_160626_2,G PMT (3)_160626_2,G PMT (4)_160626_2,G PMT (5)_160626_2,G PMT (6)_160626_2,G PMT (7)_160626_2,G PMT (8)_160626_2,G PMT (9)_160626_2,...,G PMT (17)_160626_2,G PMT (18)_160626_2,G PMT (19)_160626_2,G PMT (20)_160626_2,G PMT (21)_160626_2,G PMT (22)_160626_2,G PMT (23)_160626_2,G PMT (24)_160626_2,G PMT (25)_160626_2,G PMT (26)_160626_2
0,IAA10,1291.544864,1268.901489,1050.995830,1032.657568,804.474341,911.311318,1389.563239,925.153386,1290.568693,...,820.584477,621.505807,1068.333784,1247.801045,819.507023,776.600750,1483.688773,1220.963977,1269.520023,890.331375
1,IAA01,1281.165409,1297.232045,976.186943,1005.542795,764.650102,869.020932,1315.126239,951.213443,1572.237375,...,799.432875,589.821364,1056.008273,1137.161398,785.458284,751.138534,1411.801398,1206.080909,1168.628364,815.871295
2,AP,1376.152591,1347.562330,1137.701045,1044.827705,822.350216,930.923795,1377.323034,947.701068,1438.613318,...,834.238273,614.483818,1054.358159,1229.093091,803.986182,840.312534,1439.084477,1277.098977,1169.697193,833.271216
3,MS10,1387.064273,1321.578420,1128.520864,1127.331182,878.369170,1008.929523,1454.279205,930.380080,1345.281511,...,858.798727,671.313159,1131.200716,1222.376602,839.462682,855.376091,1502.909273,1324.046932,1216.110466,924.447307
4,IAA05,1262.019511,1267.637943,999.168159,1017.357932,785.982455,896.120761,1309.384114,940.718886,1135.399284,...,779.928852,593.644534,1040.643727,1166.757489,774.938148,750.147955,1355.461602,1162.023182,1153.194432,845.589420
5,Hexanal01,1381.141159,1334.504545,1087.204977,1064.206375,852.481955,945.278216,1507.448455,1039.246023,1766.495557,...,861.620341,624.156148,1160.438830,1254.024864,857.482239,889.155545,1670.571489,1433.757386,1245.991182,955.639045
6,EB,1351.074568,1348.900432,1099.199443,1078.011205,836.903023,927.686307,1338.786659,949.252159,1192.446295,...,855.680511,638.163864,1079.776602,1219.231080,864.064875,819.878080,1465.685727,1279.233636,1141.404034,836.367227
7,MS01,1221.955841,1317.361091,1068.941818,1013.127773,781.233273,892.717443,1362.300057,922.448602,1434.050114,...,833.584830,608.652341,1035.554773,1157.286216,815.830807,800.496727,1447.919864,1231.496250,1163.552636,855.852295
8,PA,1356.900136,1295.550489,1098.453239,1001.486989,800.393045,905.167034,1431.667841,1033.042261,1351.153682,...,821.524295,605.601545,1158.286091,1231.370250,794.850693,817.811818,1441.427114,1290.111591,1149.943580,854.777102
9,MS05,1293.685966,1277.941136,980.143170,1022.320773,829.538557,912.101295,1306.427966,923.909693,1255.346307,...,792.268693,606.287659,1032.395341,1107.825239,838.414045,784.148602,1413.797330,1190.936477,1145.353318,822.222693


In [69]:
pd.read_csv('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\GoodFiles\\160325_1\\58dt.txt').size
# badfile=pd.read_table('C:\\Users\\Annie\\Documents\\Data\\Ca_Imaging\\Untrended_For_B\\160401_1\\50.txt',skiprows=4)
# rightfile=badfile.iloc[:,3:18]
# rightfile
# rightfile.to_csv('44new.csv',index=False)

IOError: File C:\Users\Annie\Documents\Data\Ca_Imaging\GoodFiles\160325_1\58dt.txt does not exist

In [37]:
meanbase

NameError: name 'meanbase' is not defined